In [1]:
# read the data file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[idx] for idx in l])

print(encode('hii there\no'))
print(decode(encode('hii there\no')))

[46, 47, 47, 1, 58, 46, 43, 56, 43, 0, 53]
hii there
o


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [5]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")



when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [7]:
batch_size = 4 # number of independent sequences will be processed in parallel
block_size = 8 # Maximum context length for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

print('input:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()}, the target: {target}")



input:
torch.Size([4, 8])
tensor([[ 6,  1, 51, 63,  1, 50, 53, 56],
        [46, 47, 57,  1, 50, 47, 49, 47],
        [21,  1, 46, 39, 60, 43,  1, 57],
        [42,  1, 52, 53, 58,  1, 47, 52]])
target:
torch.Size([4, 8])
tensor([[ 1, 51, 63,  1, 50, 53, 56, 42],
        [47, 57,  1, 50, 47, 49, 47, 52],
        [ 1, 46, 39, 60, 43,  1, 57, 39],
        [ 1, 52, 53, 58,  1, 47, 52,  1]])
-----
When input is [6], the target: 1
When input is [6, 1], the target: 51
When input is [6, 1, 51], the target: 63
When input is [6, 1, 51, 63], the target: 1
When input is [6, 1, 51, 63, 1], the target: 50
When input is [6, 1, 51, 63, 1, 50], the target: 53
When input is [6, 1, 51, 63, 1, 50, 53], the target: 56
When input is [6, 1, 51, 63, 1, 50, 53, 56], the target: 42
When input is [46], the target: 47
When input is [46, 47], the target: 57
When input is [46, 47, 57], the target: 1
When input is [46, 47, 57, 1], the target: 50
When input is [46, 47, 57, 1, 50], the target: 47
When input is [46, 4

In [8]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

class BigramLanguageMode(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) 
            # focus only on the last time step
            logits = logits[:, -1, :] #(B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

m = BigramLanguageMode(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100).shape)
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7314, grad_fn=<NllLossBackward0>)
torch.Size([1, 101])

q$RtibytM vgIFlLtAXLPJor FsHyHM?LrErf?emF
DnQN
q3uemeB,Es?KI, nH3I,Ekw-Nfhx&is:$OPUjFYcmfI!Lvbo
qIX,


In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [12]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

/home/nguyen-van-anh/anaconda3/envs/mit_dl/lib/python3.11/site-packages/torch/autograd/graph.py:824: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


2.4605743885040283


In [13]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))





Marutevespalighat me softhiere!
Mysove wig? qThal,
's spal'lon!

NG llorincangodathiler ffo dfourintothout d , ereve nd y espaspllas hrindef nd ad t,
S:
BEN waved Ge tholayorqKE:
He,

S$jud maved brthache Clixtoling pepueramashe o, houcthitowo mer r ame veanit owph then h ay cothe CLBe oous athigalermarercFI wou ys iot GLje upe keedy, at y t d ssthe,
Jad?
Foromfif f speth.
ARKESends my mys f lanfal wnThomeand l h, ot w usin m w t s hereveak sine.
Ho atosal,
POfoveger w, she weadare h hare thil


In [12]:
# take mean of previous token embedding
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b # the mean of previous token
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[5., 4.],
        [6., 9.],
        [7., 2.]])
tensor([[5.0000, 4.0000],
        [5.5000, 6.5000],
        [6.0000, 5.0000]])


In [13]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)


In [14]:
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow = wei @ x # (T,T) @ (B,T,C) -> (B,T,T) @ (B,T,C) (broadcasting) = (B,T,C) ((T,T)@(T,C) = (T,C))
print(xbow)

tensor([[[ 1.3960e-01, -1.3628e-01],
         [-6.8228e-02,  1.7091e-01],
         [ 3.8014e-01,  9.3033e-01],
         [ 3.6535e-01,  2.8375e-01],
         [ 6.1040e-01,  2.9287e-01],
         [ 5.0342e-01,  3.2057e-01],
         [ 6.1754e-01,  9.7736e-02],
         [ 5.0634e-01,  8.7111e-02]],

        [[ 3.6752e-01,  1.6582e+00],
         [ 6.7089e-01,  8.0709e-01],
         [ 5.5537e-01,  4.9290e-01],
         [ 5.0720e-01,  3.8282e-01],
         [-6.4576e-02,  4.5483e-01],
         [-1.3706e-01,  5.2291e-01],
         [-2.8387e-06,  2.1628e-01],
         [-2.9830e-02,  3.1104e-01]],

        [[ 5.0753e-01,  2.7603e-01],
         [ 2.5183e-01,  5.7076e-01],
         [ 4.4268e-01,  2.6621e-01],
         [-6.8262e-02,  6.6389e-01],
         [-4.4772e-01,  9.1569e-01],
         [-4.9441e-01,  6.1010e-01],
         [-5.5093e-01,  2.7498e-01],
         [-5.6589e-01, -9.9914e-02]],

        [[ 3.9319e-01, -7.9836e-01],
         [-3.7638e-01, -1.6857e+00],
         [-6.0667e-01, -6.3094e-

In [16]:
# another version of this solution
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow2 = wei@x
torch.allclose(xbow, xbow2)



True

In [ ]:
# self-masked attention
B,T,C = 4,8,32 # Batch, time, channels
x = torch.randn((B,T,C))
# a single Head performs self-masked attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
querry = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,head_size)
q = querry(x) # (B,T,head_size)
wei = q @ k.transpose(-2, -1) # (B,T,head_size) -> (B,head_size,T)

tril = torch.tril(torch.ones((T,T)))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v # (B,T,head_size)

wei[0]

tensor([[0.0382, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3925, 0.2344, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0628, 0.0897, 0.1204, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0861, 0.0320, 0.2115, 0.4392, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1266, 0.5072, 0.1408, 0.0614, 0.6266, 0.0000, 0.0000, 0.0000],
        [0.1334, 0.0168, 0.0784, 0.2927, 0.2358, 0.1020, 0.0000, 0.0000],
        [0.0730, 0.0386, 0.2934, 0.1615, 0.0814, 0.6276, 0.4576, 0.0000],
        [0.0875, 0.0812, 0.1554, 0.0451, 0.0562, 0.2704, 0.5424, 1.0000]],
       grad_fn=<SelectBackward0>)

In [24]:
key.weight.shape

torch.Size([16, 32])

Self atention: keys and values are produced from the same source as queries.